In [8]:
import numpy as np
# importing decimal module to control significant figures
import decimal

In [9]:
def toDecimal(array) :
    # flatten the array convert floats to decimal and reshape to original shape
    originalShape = array.shape
    array = array.flatten().astype(object) 
    for i in range(array.size) :
        array[i] = decimal.Decimal(str(array[i]))
    array = array.reshape(originalShape)   
    return array


def intializeContext(significantFigs,rounding) :
    #Intialize the context with specified signFigs and rounding/chopping
    decimal.getcontext().prec = significantFigs 
    if( rounding ) :
        decimal.getcontext().rounding = decimal.ROUND_HALF_UP
    else :
        decimal.getcontext().rounding = decimal.ROUND_DOWN 
        
def toFloats (array) :
    #Convert the decimals back to float
    originalShape = array.shape
    array = array.flatten().astype(object)
    for i in range(array.size) :
        array[i] = float(array[i]) 
    array = array.reshape(originalShape)    
    return array            
    

In [16]:
def isDiagonalyDominant(A) :
    #Check if Matrix A is diagonaly dominant 
    dim = A.shape
    # Flags to check system
    at_least_one_strictly_greater = None 
    if(dim[0] != dim[1]) : raise Exception("Matrix must be square")
    for i in range(dim[0]) :
        nonDiagonalSum = 0 
        for j in range(dim[0]) :
            if(j==i) : continue 
            nonDiagonalSum += abs(A[i][j]) 
        if(A[i][i] > nonDiagonalSum) :
            at_least_one_strictly_greater = True
        elif(A[i][i] == nonDiagonalSum) :
            if(at_least_one_strictly_greater == None) : at_least_one_strictly_greater = False
        else :
            return False    
    return at_least_one_strictly_greater            
            

In [10]:
# Jacobi with relaxation for improved convergence (called weighted Jacobi)
# the parameter n can be removed as we can get the no of vars from the npArray
def Jacobi(A,b,n,x,maxIterations,ErrorTolerance,relax , significantFigs = 7 , rounding = True) :
    # Setting up signifcant figs and rounding/chopping
    intializeContext(significantFigs,rounding)
    #Copying the arrays to avoid modifying original arrays
    A = A.copy()
    b = b.copy()
    x = x.copy()
    #converting floats to decimals 
    A = toDecimal(A)
    b = toDecimal(b)
    x = toDecimal(x)
    relax = decimal.Decimal(str(relax))
    #Array to hold new values
    xNew = np.zeros(n,dtype=object)
    # Normalizaing the equations such that a(i,i) = 1
    for i in range(n) :
        diag = A[i][i]
        for j in range(n) :
            A[i][j] = A[i][j] / diag
        b[i] = b[i] / diag
    # Calculating first iteration before applying relaxation    
    for i in range(n) :
        sum = b[i]
        for j in range(n) :
            if(i==j) :
                continue
            sum -= A[i][j] * x[j]
        xNew[i] = sum 
    iteration = 1
    # Loop until convergence or max iterations reached 
    while (True) :
        belowTolerance = True
        for i in range(n) :
            oldX = x[i]
            sum = b[i]
            for j in range(n) :
                if(i==j) :
                    continue
                sum -= A[i][j] * x[j]
            xNew[i] = relax*sum + (1-relax)*oldX
            if (belowTolerance and xNew[i] != 0) :
                estimatedError = abs((xNew[i]-oldX)/xNew[i])
                estimatedError = float(estimatedError)
                if(estimatedError > ErrorTolerance):
                    belowTolerance = False
        iteration+=1
        x = xNew.copy()
        if(belowTolerance or iteration >= maxIterations) :
            break
    return toFloats(xNew)

In [11]:

def run_test(test_name, A, b, x0, relax=1.0, maxIters=100, tol=1e-6):
    print("==============================================")
    print(f"TEST: {test_name}")
    print("----------------------------------------------")
    print("Method: Jacobi (relax =", relax, ")")

    # Ensure inputs are NumPy arrays
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    x0 = np.array(x0, dtype=float)

    # True solution
    true_solution = np.linalg.solve(A, b)
    print("True solution:", true_solution)

    # Run Jacobi
    sol = Jacobi(A.copy(), b.copy(), len(b), x0.copy(),
                 maxIterations=maxIters,
                 ErrorTolerance=tol,
                 relax=relax)

    print("Jacobi solution:", np.array(sol, dtype=float))
    print("==============================================\n")


In [12]:
A = [[4, -1],
     [-1, 3]]

b = [9, 3]
x0 = [0, 0]

run_test("Simple 2x2 System", A, b, x0)

A = [
    [10, 1, 1],
    [2, 10, 1],
    [2, 2, 10]
]

b = [12, 13, 14]
x0 = [0, 0, 0]

run_test("3x3 Diagonally Dominant System", A, b, x0)

run_test("Weighted Jacobi (relax=0.5)", A, b, x0, relax=0.5, maxIters=200)

run_test("Weighted Jacobi (relax=1.2)", A, b, x0, relax=1.2, maxIters=200)

A = [[2, 3],
     [3, 2]]

b = [5, 5]
x0 = [0, 0]

run_test("Non-Diagonally Dominant System", A, b, x0)




TEST: Simple 2x2 System
----------------------------------------------
Method: Jacobi (relax = 1.0 )
True solution: [2.72727273 1.90909091]
Jacobi solution: [2.727273 1.909091]

TEST: 3x3 Diagonally Dominant System
----------------------------------------------
Method: Jacobi (relax = 1.0 )
True solution: [1. 1. 1.]
Jacobi solution: [1. 1. 1.]

TEST: Weighted Jacobi (relax=0.5)
----------------------------------------------
Method: Jacobi (relax = 0.5 )
True solution: [1. 1. 1.]
Jacobi solution: [0.9999992 1.000001  1.000001 ]

TEST: Weighted Jacobi (relax=1.2)
----------------------------------------------
Method: Jacobi (relax = 1.2 )
True solution: [1. 1. 1.]
Jacobi solution: [1. 1. 1.]

TEST: Non-Diagonally Dominant System
----------------------------------------------
Method: Jacobi (relax = 1.0 )
True solution: [1. 1.]
Jacobi solution: [2.710431e+17 2.710431e+17]



In [13]:
A = [[4, 1],
     [2, 3]]

b = [0, 0]
x0 = [0, 0]

run_test("Zero RHS System", A, b, x0)


TEST: Zero RHS System
----------------------------------------------
Method: Jacobi (relax = 1.0 )
True solution: [0. 0.]
Jacobi solution: [0. 0.]



In [14]:
np.random.seed(0)
A = np.random.randint(1, 5, (10, 10)).astype(float)

# Make diagonally dominant
for i in range(10):
    A[i][i] = sum(abs(A[i])) + 1

b = np.random.randint(1, 10, 10).tolist()
x0 = np.zeros(10).tolist()

run_test("10x10 Large System", A.tolist(), b, x0)


TEST: 10x10 Large System
----------------------------------------------
Method: Jacobi (relax = 1.0 )
True solution: [0.05969281 0.14064141 0.19486967 0.06476413 0.06019018 0.06553446
 0.10112902 0.12167262 0.24724811 0.06096309]
Jacobi solution: [0.05969297 0.1406415  0.1948699  0.06476438 0.06019027 0.06553461
 0.1011293  0.1216727  0.2472484  0.06096321]



In [15]:
def test_diagonal_dominance(A, expected_result, test_name):
    try:
        result = isDiagonalyDominant(A)
        print(f"{test_name}: {'PASS' if result == expected_result else 'FAIL'} | Expected: {expected_result}, Got: {result}")
    except Exception as e:
        print(f"{test_name}: Exception occurred - {e}")

In [17]:
A1 = np.array([[5, 1, 1],
               [2, 6, 1],
               [1, 2, 7]])
test_diagonal_dominance(A1, True, "Fully Strict Dominant")


Fully Strict Dominant: PASS | Expected: True, Got: True


In [21]:
A2 = np.array([[4, 1, 2],
               [1, 4, 3],
               [2, 2, 4]])  # last row weak (diagonal = sum of off-diagonals)
test_diagonal_dominance(A2, True, "Mixed Strict + Weak")


Mixed Strict + Weak: PASS | Expected: True, Got: True


In [19]:
A3 = np.array([[2, 1, 1],
               [1, 2, 1],
               [1, 1, 2]])
test_diagonal_dominance(A3, False, "All Weak, No Strict")


All Weak, No Strict: PASS | Expected: False, Got: False


In [22]:
A4 = np.array([[1, 2, 1],
               [2, 1, 2],
               [1, 2, 1]])
test_diagonal_dominance(A4, False, "Not Dominant")


Not Dominant: PASS | Expected: False, Got: False


In [23]:
A6 = np.array([[1, 2, 3],
               [4, 5, 6]])
test_diagonal_dominance(A6, None, "Non-square Matrix")


Non-square Matrix: Exception occurred - Matrix must be square


In [24]:
A5 = np.array([[5]])
test_diagonal_dominance(A5, True, "Single Element")


Single Element: PASS | Expected: True, Got: True
